In [4]:
%run "libraries.ipynb"
import networkx as nx
from IPython.html import widgets
from IPython.display import display, HTML

# importing datasets

In [5]:
# list of page names
pages = codecs.open("data/pagenames.txt","r", "utf-8-sig").readlines()
pages = map(lambda x: x.strip(), pages)

# page graph obtained by projecting page-editor bi-partite graph
pages_graph = nx.read_gexf("data/pages-linked-by-coeditors.gexf")

In [6]:
def table_to_html(data, cols=[]):
    html = "<table>"
    html += "<tr>"
    for column_content in cols:
        html += "<th>%s</th>" % (column_content)
    html += "</tr>"
    
    for d in data:
        html += "<tr>"
        for column_content in d:
            html += "<td>%s</td>" % (column_content)
        html += "</tr>"
    html += "</table>"
        
    return HTML(html) 

def show_info(page):
    display(HTML("<h2>%s</h2>" % (page)))
    
    #display(HTML("<div style=\"float:left\">"))    
    display(HTML("<h3>co-edited pages</h3>"))

    nb = sorted(pages_graph["p:%s" % (page)].items(),
            key=lambda (k,x): -int(x["coeditors"]))

    data = []

    # calculate rank in neighbor top co-edited ranking
    for name, info in nb:
        nb_mirror = sorted(pages_graph[name].items(),
                key=lambda (k,x): -int(x["coeditors"]))
        nb_mirror = [ x[0] for x in nb_mirror ]

        info["ranking"] = nb_mirror.index("p:%s" % (page)) + 1

    #print nb

    for name, info in nb[0:10]:
        data.append([ name.split(":")[1],  info["coeditors"], info["ranking"]])
        
    display(table_to_html(data, ["page name", "co-editors", "ranking"]))
    #display(HTML("</div>"))    

    #display(HTML("<div style=\"float:left\">"))    
    display(HTML("<h3>ranked first in</h3>"))

    nb_list = [ x[0] for x in nb ]
    data = []

    nb2 = sorted(nb, key=lambda (x): x[1]["ranking"])
    for name, info in nb2[0:10]:
        data.append([ name.split(":")[1],  info["coeditors"], info["ranking"]])    
    display(table_to_html(data, ["page name", "co-editors", "ranking"]))
#     display(HTML("</div>"))    
#     display(HTML("<div style=\"clear:both\"></div>"))
    
    display(HTML("<h3>pageviews and revisions</h3>"))
    display(HTML("<h3>top editors</h3>"))
    
# show_info("Pi")

# explorer

In [7]:
w = widgets.SelectWidget(description="page", options=pages)
widgets.interact(show_info, page=w);

page name,co-editors,ranking
Pi,24,168
Pythagorean theorem,24,143
Pythagorean triple,21,83
Triangle,20,152
Polygon,20,140
Circle,19,157
Tetrahedron,19,145
Equilateral triangle,18,72
Polyhedron,18,152
Mathematics,17,191


page name,co-editors,ranking
Integer triangle,10,2
Heronian tetrahedron,7,2
Dissection problem,6,14
List of triangle topics,10,29
Orthocentric system,11,33
Holditch's theorem,8,38
Pedal triangle,7,40
Symmedian,13,42
Equidiagonal quadrilateral,3,44
Hadwiger's theorem,6,47
